# Feature Engineering

These techniques are adapted from the ones covered in the [Feature Engineering](https://www.kaggle.com/learn/feature-engineering) course on Kaggle.

In [1]:
# Global variables for testing changes to this notebook quickly
FOLD_SEED = 0
NUM_FOLDS = 3
EARLY_STOP = 50
TRIALS = 100

In [2]:
# Essentials
import os
import warnings
import numpy as np
import pandas as pd
import time

# Preprocessing
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, KFold
from functools import partial 
from sklearn.impute import SimpleImputer, KNNImputer
from category_encoders import OrdinalEncoder, OneHotEncoder

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from category_encoders import MEstimateEncoder

# Models
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Hyperparameter Tuning
import optuna
from optuna.samplers import GridSampler
from optuna.visualization import plot_parallel_coordinate

# Mute warnings
warnings.filterwarnings('ignore')

# Create Folds and Preprocessing

In [3]:
# Load the training data
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

# Remove rows with missing target
train.dropna(axis=0, subset=['SalePrice'], inplace=True)

# Clean data, static transformations
def clean_data(*data):
    for df in data:
        # fix typos to match documentation
        df['MSZoning'] =  df['MSZoning'].replace({'C (all)': 'C'})
        df["Exterior2nd"] = df["Exterior2nd"].replace({"Brk Cmn":"BrkComm","Wd Shng": "WdShing"})
        df['Neighborhood'] = df['Neighborhood'].replace({'NAmes':'Names'})

        # Some values of GarageYrBlt are corrupt, replace them with YearBuilt
        df["GarageYrBlt"] = df["GarageYrBlt"].where(df.GarageYrBlt <= 2010, df.YearBuilt)
        
        # optional feature: A - agriculture, C - commercial, R - residential, I - industrial
        df["MSClass"] = df['MSZoning'].map({'A': 'A','C': 'C',"FV": 'R','I': 'I',
                                            "RH": 'R',"RL": 'R',"RP": 'R',"RM": 'R', np.nan:np.nan})
    return data
    
train, test = clean_data(train, test)

# List of categorical/numerical columns
columns = [col for col in test.columns if col not in ["Id","MSClass"]]
object_cols = [col for col in columns if train[col].dtype == "object"]
number_cols = [col for col in columns if train[col].dtype != "object"]

# Define bins
binner = KBinsDiscretizer(n_bins = 45, encode = 'ordinal', strategy = 'uniform')
y_bins = binner.fit_transform(pd.DataFrame(data=train['SalePrice']))

# Define folds
train["kfold"] = -1
kf = StratifiedKFold(NUM_FOLDS, shuffle = True, random_state = FOLD_SEED) 
for fold, (train_idx, valid_idx) in enumerate(kf.split(train, y_bins)):
    train.loc[valid_idx,"kfold"] = fold

In [4]:
def preprocessing(X_train, X_valid, X_test):
    
    # 1. impute numerical data
    columns = [col for col in X_train.columns if X_train[col].dtype != "object"]
    if columns:
        imputer = SimpleImputer(strategy='mean')
        X_train[columns] = imputer.fit_transform(X_train[columns])
        X_valid[columns] = imputer.transform(X_valid[columns])
        X_test[columns] = imputer.transform(X_test[columns])
    
    # 2. impute categorical data
    columns = [col for col in X_train.columns if X_train[col].dtype == "object"]
    if columns:
        imputer = SimpleImputer(strategy='constant', fill_value = 'None')
        X_train[columns] = imputer.fit_transform(X_train[columns])
        X_valid[columns] = imputer.transform(X_valid[columns])
        X_test[columns] = imputer.transform(X_test[columns])
    
    # 3. encode 1-10 ratings
    cols = ["OverallQual","OverallCond"]
    cols = [x for x in cols if x in X_train.columns]
    ratings = {float(a):b for b,a in enumerate(range(1,11))}
    mapping = [{'col':x, 'mapping': ratings} for x in cols]
    
    encoder = OrdinalEncoder(cols = cols, mapping = mapping)
    X_train = encoder.fit_transform(X_train)
    X_valid = encoder.transform(X_valid)
    X_test = encoder.transform(X_test)
    
    # 4. encode Poor, Fair, Avg, Good, Ex ratings
    cols = ["ExterQual","ExterCond","BsmtQual","BsmtCond","HeatingQC", "KitchenQual","FireplaceQu","GarageQual","GarageCond",'PoolQC']
    cols = [x for x in cols if x in X_train.columns]
    ratings = {"Po":0, "Fa":1, "TA":2, "Gd":3, "Ex":4}
    mapping = [{'col':x, 'mapping': ratings} for x in cols]
    
    encoder = OrdinalEncoder(cols = cols, mapping = mapping)
    X_train = encoder.fit_transform(X_train)
    X_valid = encoder.transform(X_valid)
    X_test = encoder.transform(X_test)
    
    # 5. encode remaining ordinal data
    cols = ["LotShape","LandSlope","BsmtExposure","BsmtFinType1","BsmtFinType2",
    "Functional","GarageFinish","PavedDrive","Utilities","CentralAir","Electrical",
    "Fence"]
    cols = [x for x in cols if x in X_train.columns]
    mapping = [{'col':"LotShape",
                'mapping': {"Reg":0, "IR1":1, "IR2":2, "IR3":3}},
               {'col':"LandSlope",
                'mapping': {"Sev":0, "Mod":1, "Gtl":2}},
               {'col':"BsmtExposure",
                'mapping': {"No":0, "Mn":1, "Av":2, "Gd":3}},
               {'col':"BsmtFinType1",
                'mapping': {"Unf":0, "LwQ":1, "Rec":2, "BLQ":3, "ALQ":4, "GLQ":5}},
               {'col':"BsmtFinType2",
                'mapping': {"Unf":0, "LwQ":1, "Rec":2, "BLQ":3, "ALQ":4, "GLQ":5}},
               {'col':"Functional",
                'mapping': {"Sal":0, "Sev":1, "Maj1":2, "Maj2":3, "Mod":4, "Min2":5, "Min1":6, "Typ":7}},
               {'col':"GarageFinish",
                'mapping': {"Unf":0, "RFn":1, "Fin":2}},
               {'col':"PavedDrive",
                'mapping': {"N":0, "P":1, "Y":2}},
               {'col':"Utilities",
                'mapping': {"NoSeWa":0, "NoSewr":1, "AllPub":2}},
               {'col':"CentralAir",
                'mapping': {"N":0, "Y":1}},
               {'col':"Electrical",
                'mapping': {"Mix":0, "FuseP":1, "FuseF":2, "FuseA":3, "SBrkr":4}},
               {'col':"Fence",
                'mapping': {"MnWw":0, "GdWo":1, "MnPrv":2, "GdPrv":3}}]
    mapping = [x for x in mapping if x['col'] in X_train.columns]
    
    encoder = OrdinalEncoder(cols = cols, mapping = mapping)
    X_train = encoder.fit_transform(X_train)
    X_valid = encoder.transform(X_valid)
    X_test = encoder.transform(X_test)
    
    # Determine cardinality of remaining categorical data
    columns = [col for col in X_train.columns if X_train[col].dtype == 'object']
    high_cols = [col for col in columns if X_train[col].nunique() >= 10]
    low_cols = [col for col in columns if X_train[col].nunique() < 10]
    
    # 6. ordinal encode high cardinality data
    if high_cols:
        encoder = OrdinalEncoder(cols = high_cols)
        X_train = encoder.fit_transform(X_train)
        X_valid = encoder.transform(X_valid)
        X_test = encoder.transform(X_test)
    
    # 7. one-hot encode low cardinality data
    if low_cols:
        encoder = OneHotEncoder(cols = low_cols, use_cat_names = True)
        X_train = encoder.fit_transform(X_train)
        X_valid = encoder.transform(X_valid)
        X_test = encoder.transform(X_test)
        
    return X_train, X_valid, X_test

# Scoring Functions

In [5]:
def score_xgboost(transforms = [preprocessing], params = {}, cols = columns, verbose = True):
    start = time.time()
    
    X = train.copy()
    scores = np.zeros(NUM_FOLDS)
    
    for i in range(NUM_FOLDS):
        X_train = X[X.kfold != i][cols].reset_index(drop=True)
        X_valid = X[X.kfold == i][cols].reset_index(drop=True)
        y_train = X[X.kfold != i]['SalePrice'].reset_index(drop=True)
        y_valid = X[X.kfold == i]['SalePrice'].reset_index(drop=True)
        X_test = test[cols].copy()
        
        # loop for applying the transformations
        for transform in transforms:
            try:
                X_train, X_valid, X_test = transform(X_train, X_valid, X_test, y_train = y_train)
            except:
                X_train, X_valid, X_test = transform(X_train, X_valid, X_test)
        
        model = XGBRegressor(**{**{'random_state':0, 'n_estimators': 3000},**params})
        model.fit(X_train, y_train,
                  verbose = False,
                  eval_set = [(X_valid, y_valid)],
                  eval_metric = "mae",
                  early_stopping_rounds = EARLY_STOP)

        preds = model.predict(X_valid)
        scores[i] = mean_absolute_error(y_valid, preds)
    end = time.time()
    if verbose:
        print("XGBoost  (3-fold Avg):", 
              round(scores.mean(), 4))
        print("XGBoost  (3-fold Max):", 
              round(scores.max(), 4), "\t",
              str(round(end-start, 3))+"s")

    return round(scores.mean(), 4), round(scores.max(), 4)

In [6]:
def score_lightgbm(transforms = [preprocessing], params = {}, cols = columns, verbose = True):
    start = time.time()
    
    X = train.copy()
    scores = np.zeros(NUM_FOLDS)
    
    for i in range(NUM_FOLDS):
        X_train = X[X.kfold != i][cols].reset_index(drop=True)
        X_valid = X[X.kfold == i][cols].reset_index(drop=True)
        y_train = X[X.kfold != i]['SalePrice'].reset_index(drop=True)
        y_valid = X[X.kfold == i]['SalePrice'].reset_index(drop=True)
        X_test = test[cols].copy()
        
        for transform in transforms:
            try:
                X_train, X_valid, X_test = transform(X_train, X_valid, X_test, y_train = y_train)
            except:
                X_train, X_valid, X_test = transform(X_train, X_valid, X_test)
                
        cat_cols = [x for x in X_train.columns if x in object_cols]
                
        model = LGBMRegressor(**{**{'random_state':0, 'n_estimators': 3000},**params})
        model.fit(X_train, y_train,
                  verbose = False,
                  eval_set = [(X_valid, y_valid)],
                  eval_metric = "mae",
                  categorical_feature = cat_cols,
                  early_stopping_rounds = EARLY_STOP)

        valid_preds = model.predict(X_valid)
        scores[i] = mean_absolute_error(y_valid, valid_preds)
    end = time.time()
    if verbose:
        print("LightGBM (3-fold Avg):", 
              round(scores.mean(), 4))
        print("LightGBM (3-fold Max):", 
              round(scores.max(), 4), "\t",
              str(round(end-start, 3))+"s")

    return round(scores.mean(), 4), round(scores.max(), 4)

In [7]:
def score_catboost(transforms = [preprocessing], params = {}, cols = columns, verbose = True):
    start = time.time()
    
    X = train.copy()
    scores = np.zeros(NUM_FOLDS)
    
    for i in range(NUM_FOLDS):
        X_train = X[X.kfold != i][cols].reset_index(drop=True)
        X_valid = X[X.kfold == i][cols].reset_index(drop=True)
        y_train = X[X.kfold != i]['SalePrice'].reset_index(drop=True)
        y_valid = X[X.kfold == i]['SalePrice'].reset_index(drop=True)
        X_test = test[cols].copy()
        
        # loop for applying the transformations
        for transform in transforms:
            try:
                X_train, X_valid, X_test = transform(X_train, X_valid, X_test, y_train = y_train)
            except:
                X_train, X_valid, X_test = transform(X_train, X_valid, X_test)
        
        model = CatBoostRegressor(**{**{'random_state':0, 
                                        'n_estimators': 3000,
                                        'eval_metric':"MAE",
                                        'early_stopping_rounds': EARLY_STOP,
                                        'verbose': False}, **params})
        model.fit(X_train, y_train,
                  eval_set = (X_valid, y_valid),
                  use_best_model=True)

        valid_preds = model.predict(X_valid)
        scores[i] = mean_absolute_error(y_valid, valid_preds)
    end = time.time()
    if verbose:
        print("CatBoost (3-fold Avg):", 
              round(scores.mean(), 4))
        print("CatBoost (3-fold Max):", 
              round(scores.max(), 4), "\t",
              str(round(end-start, 3))+"s")

    return round(scores.mean(), 4), round(scores.max(), 4)

In [8]:
def get_baseline():
    
    print("\nBaseline\n")
    xgb_avg, xgb_max = score_xgboost()
    lgbm_avg, lgbm_max = score_lightgbm()
    cat_avg, cat_max = score_catboost()
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
    
    return round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4), round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4)
    
BASELINE_AVG, BASELINE_MAX = get_baseline()


Baseline

XGBoost  (3-fold Avg): 17818.5785
XGBoost  (3-fold Max): 18420.5786 	 2.004s
LightGBM (3-fold Avg): 16481.3275
LightGBM (3-fold Max): 17172.4876 	 1.665s
CatBoost (3-fold Avg): 15152.8277
CatBoost (3-fold Max): 16049.8435 	 12.3s

Overall (Avg):         16484.2446
Overall (Max):         17818.5785


# Mutual Information

In [9]:
def remove_uninformative(X_train, X_valid, X_test, y_train, verbose = False):
    
    # 1. Determine uninformative columns
    scores =  mutual_info_regression(X_train, y_train)
    cols = [x for i, x in enumerate(X_train.columns) if scores[i] == 0]
    
    # 2. Drop the uninformative columns
    X_train.drop(cols, axis = 1, inplace = True)
    X_valid.drop(cols, axis = 1, inplace = True)
    X_test.drop(cols, axis = 1, inplace = True)
    
    if verbose:
        print("Dropped columns:", *cols)
    
    return X_train, X_valid, X_test

def test_uninformative():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX)
    
    print("\nDrop Uninformative\n")
    transforms = [preprocessing, remove_uninformative]
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
    
test_uninformative()


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

Drop Uninformative

XGBoost  (3-fold Avg): 17908.9726
XGBoost  (3-fold Max): 18301.5848 	 4.426s
LightGBM (3-fold Avg): 16625.3983
LightGBM (3-fold Max): 17315.2156 	 3.986s
CatBoost (3-fold Avg): 15054.8003
CatBoost (3-fold Max): 15586.0086 	 16.549s

Overall (Avg):         16529.7237
Overall (Max):         17908.9726


# Mathematical Transformations

In [10]:
def mathematical_transformations(X_train, X_valid, X_test):
    
    X_train["LivLotRatio"] = X_train["GrLivArea"] / X_train["LotArea"]
    X_valid["LivLotRatio"] = X_valid["GrLivArea"] / X_valid["LotArea"]
    X_test["LivLotRatio"] = X_test["GrLivArea"] / X_test["LotArea"]
    
    X_train["Spaciousness"] = (X_train["1stFlrSF"]+X_train["2ndFlrSF"]) / X_train["TotRmsAbvGrd"]
    X_valid["Spaciousness"] = (X_valid["1stFlrSF"]+X_valid["2ndFlrSF"]) / X_valid["TotRmsAbvGrd"]
    X_test["Spaciousness"] = (X_test["1stFlrSF"]+X_test["2ndFlrSF"]) / X_test["TotRmsAbvGrd"]
    
    X_train["TotalOutsideSF"] = X_train["WoodDeckSF"] + X_train["OpenPorchSF"] + X_train["EnclosedPorch"] + X_train["3SsnPorch"] + X_train["ScreenPorch"]
    X_valid["TotalOutsideSF"] = X_valid["WoodDeckSF"] + X_valid["OpenPorchSF"] + X_valid["EnclosedPorch"] + X_valid["3SsnPorch"] + X_valid["ScreenPorch"]
    X_test["TotalOutsideSF"] = X_test["WoodDeckSF"] + X_test["OpenPorchSF"] + X_test["EnclosedPorch"] + X_test["3SsnPorch"] + X_test["ScreenPorch"]
    
    X_train['TotalLot'] = X_train['LotFrontage'] + X_train['LotArea']
    X_valid['TotalLot'] = X_valid['LotFrontage'] + X_valid['LotArea']
    X_test['TotalLot'] = X_test['LotFrontage'] + X_test['LotArea']
    
    X_train['TotalBsmtFin'] = X_train['BsmtFinSF1'] + X_train['BsmtFinSF2']
    X_valid['TotalBsmtFin'] = X_valid['BsmtFinSF1'] + X_valid['BsmtFinSF2']
    X_test['TotalBsmtFin'] = X_test['BsmtFinSF1'] + X_test['BsmtFinSF2']
    
    X_train['TotalSF'] = X_train['TotalBsmtSF'] + X_train['2ndFlrSF'] + X_train['1stFlrSF']
    X_valid['TotalSF'] = X_valid['TotalBsmtSF'] + X_valid['2ndFlrSF'] + X_valid['1stFlrSF']
    X_test['TotalSF'] = X_test['TotalBsmtSF'] + X_test['2ndFlrSF'] + X_test['1stFlrSF']
    
    X_train['TotalBath'] = X_train['FullBath'] + X_train['HalfBath'] * 0.5 + X_train['BsmtFullBath'] + X_train['BsmtHalfBath'] * 0.5
    X_valid['TotalBath'] = X_valid['FullBath'] + X_valid['HalfBath'] * 0.5 + X_valid['BsmtFullBath'] + X_valid['BsmtHalfBath'] * 0.5
    X_test['TotalBath'] = X_test['FullBath'] + X_test['HalfBath'] * 0.5 + X_test['BsmtFullBath'] + X_test['BsmtHalfBath'] * 0.5
    
    X_train['TotalPorch'] = X_train['OpenPorchSF'] + X_train['EnclosedPorch'] + X_train['ScreenPorch'] + X_train['WoodDeckSF']
    X_valid['TotalPorch'] = X_valid['OpenPorchSF'] + X_valid['EnclosedPorch'] + X_valid['ScreenPorch'] + X_valid['WoodDeckSF']
    X_test['TotalPorch'] = X_test['OpenPorchSF'] + X_test['EnclosedPorch'] + X_test['ScreenPorch'] + X_test['WoodDeckSF']
    
    return X_train, X_valid, X_test

def test_transformations():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX)
    
    print("\nMathematical Transformations\n")
    transforms = [preprocessing, mathematical_transformations]
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
    
test_transformations()
    


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

Mathematical Transformations

XGBoost  (3-fold Avg): 17238.8075
XGBoost  (3-fold Max): 17584.7709 	 1.975s
LightGBM (3-fold Avg): 16471.4805
LightGBM (3-fold Max): 17043.6677 	 1.793s
CatBoost (3-fold Avg): 14850.5287
CatBoost (3-fold Max): 15603.5834 	 12.814s

Overall (Avg):         16186.9389
Overall (Max):         17238.8075


# Encoding Interactions

In [11]:
def encode_interaction(X_train, X_valid, X_test, cat_col = "BldgType", num_col = "GrLivArea"):

    X_1 = pd.get_dummies(X_train[cat_col], prefix=cat_col)
    X_2 = pd.get_dummies(X_valid[cat_col], prefix=cat_col)
    X_3 = pd.get_dummies(X_test[cat_col], prefix=cat_col)
        
    for col in X_1.columns:
        X_train[col+"_"+num_col] = X_1[col]*X_train[num_col]
        X_valid[col+"_"+num_col] = X_2[col]*X_valid[num_col]
        X_test[col+"_"+num_col] = X_3[col]*X_test[num_col]
    
    return X_train, X_valid, X_test

def test_interaction():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX)  
    
    transforms = [encode_interaction, preprocessing]
    print("\nInteraction Term:\n")
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
    
test_interaction()


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

Interaction Term:

XGBoost  (3-fold Avg): 18108.8183
XGBoost  (3-fold Max): 18803.2203 	 2.649s
LightGBM (3-fold Avg): 16724.7195
LightGBM (3-fold Max): 17621.4513 	 1.755s
CatBoost (3-fold Avg): 15163.728
CatBoost (3-fold Max): 15694.709 	 11.808s

Overall (Avg):         16665.7553
Overall (Max):         18108.8183


# Count Features

In [12]:
def count_porch_types(X_train, X_valid, X_test):
    
    X_train["PorchTypes"] = X_train[["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"]].gt(0).sum(axis=1)
    X_valid["PorchTypes"] = X_valid[["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"]].gt(0).sum(axis=1)
    X_test["PorchTypes"] = X_test[["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"]].gt(0).sum(axis=1)
        
    return X_train, X_valid, X_test

def test_count_feature():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX)
    
    transforms = [preprocessing, count_porch_types]
    print("\nCount Feature:\n")
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
      
test_count_feature()


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

Count Feature:

XGBoost  (3-fold Avg): 17770.3514
XGBoost  (3-fold Max): 18389.5722 	 2.03s
LightGBM (3-fold Avg): 16577.4662
LightGBM (3-fold Max): 17139.9331 	 1.609s
CatBoost (3-fold Avg): 15150.8127
CatBoost (3-fold Max): 15669.522 	 11.695s

Overall (Avg):         16499.5434
Overall (Max):         17770.3514


# Break Down Feature

In [13]:
def test_breakdown():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX)
    
    print("\nBreak Down:\n")
    xgb_avg, xgb_max = score_xgboost(cols = columns + ["MSClass"])
    lgbm_avg, lgbm_max = score_lightgbm(cols = columns + ["MSClass"])
    cat_avg, cat_max = score_catboost(cols = columns + ["MSClass"])
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
      
    
test_breakdown()


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

Break Down:

XGBoost  (3-fold Avg): 17818.5785
XGBoost  (3-fold Max): 18420.5786 	 2.098s
LightGBM (3-fold Avg): 16481.3275
LightGBM (3-fold Max): 17172.4876 	 1.683s
CatBoost (3-fold Avg): 15156.8618
CatBoost (3-fold Max): 15920.7323 	 12.108s

Overall (Avg):         16485.5893
Overall (Max):         17818.5785


# Grouped Transform

In [14]:
def group_transformation(X_train, X_valid, X_test):
    
    X_train["MedNhbdLvArea"] = X_train.groupby("Neighborhood")["GrLivArea"].transform('median')
    
    # we use the medians from the training data to impute the test data
    mapping = dict()
    for x,y in zip(X_train["MedNhbdLvArea"].iteritems(), X_train['Neighborhood'].iteritems()):
        _,median_area = x
        _,nbhr = y
        if nbhr not in mapping: mapping[nbhr] = median_area
    
    X_valid["MedNhbdLvArea"] = X_valid['Neighborhood'].map(mapping)
    X_test["MedNhbdLvArea"] = X_test['Neighborhood'].map(mapping)
    
    return X_train, X_valid, X_test

def test_group():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX)
    
    transforms = [preprocessing, group_transformation]
    print("\nGroup Transformation:\n")
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
      
    
test_group()


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

Group Transformation:

XGBoost  (3-fold Avg): 17904.3209
XGBoost  (3-fold Max): 19005.0376 	 2.013s
LightGBM (3-fold Avg): 16662.3771
LightGBM (3-fold Max): 17680.7477 	 1.713s
CatBoost (3-fold Avg): 14713.3128
CatBoost (3-fold Max): 15112.7124 	 10.748s

Overall (Avg):         16426.6703
Overall (Max):         17904.3209


# Clustering (Labels)

In [15]:
def generate_cluster_labels(X_train, X_valid, X_test, name = "Area", features = ['LotArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF','GrLivArea']):
    
    # 1. normalize based on training data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train[features])
    X_valid_scaled = scaler.transform(X_valid[features])
    X_test_scaled = scaler.transform(X_test[features])
    
    # 2. create cluster labels (use predict)
    kmeans = KMeans(n_clusters = 10, n_init = 10, random_state=0)
    X_train[name + "_Cluster"] = kmeans.fit_predict(X_scaled)
    X_valid[name + "_Cluster"] = kmeans.predict(X_valid_scaled)
    X_test[name + "_Cluster"] = kmeans.predict(X_test_scaled)
         
    return X_train, X_valid, X_test

def test_cluster_labels():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX)
    
    transforms = [preprocessing, generate_cluster_labels]
    print("\nCluster Labels:\n")
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
     
test_cluster_labels()


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

Cluster Labels:

XGBoost  (3-fold Avg): 18035.7252
XGBoost  (3-fold Max): 18523.0575 	 2.765s
LightGBM (3-fold Avg): 16448.1335
LightGBM (3-fold Max): 16955.8606 	 1.816s
CatBoost (3-fold Avg): 14997.3111
CatBoost (3-fold Max): 15618.4954 	 10.332s

Overall (Avg):         16493.7233
Overall (Max):         18035.7252


# Clustering (Distances)

In [16]:
def generate_cluster_distances(X_train, X_valid, X_test, name = "Area", features = ['LotArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF','GrLivArea']):
    
    # 1. normalize based on training data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train[features])
    X_valid_scaled = scaler.transform(X_valid[features])
    X_test_scaled = scaler.transform(X_test[features])
    
    # 2. generate cluster distances (use transform)
    kmeans = KMeans(n_clusters = 10, n_init = 10, random_state=0)
    X_cd = kmeans.fit_transform(X_scaled)
    X_valid_cd = kmeans.transform(X_valid_scaled)
    X_test_cd = kmeans.transform(X_test_scaled)
    
    # 3. column labels
    X_cd = pd.DataFrame(X_cd, columns=[name + "_Centroid_" + str(i) for i in range(X_cd.shape[1])])
    X_valid_cd = pd.DataFrame(X_valid_cd, columns=[name + "_Centroid_" + str(i) for i in range(X_valid_cd.shape[1])])
    X_test_cd = pd.DataFrame(X_test_cd, columns=[name + "_Centroid_" + str(i) for i in range(X_test_cd.shape[1])])    
    
    return X_train.join(X_cd), X_valid.join(X_valid_cd), X_test.join(X_test_cd)

def test_cluster_distances():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX)
    
    transforms = [preprocessing, generate_cluster_distances]
    print("\nCluster Distances:\n")
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
     
    
test_cluster_distances()


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

Cluster Distances:

XGBoost  (3-fold Avg): 17902.8628
XGBoost  (3-fold Max): 18883.7932 	 2.393s
LightGBM (3-fold Avg): 16404.7344
LightGBM (3-fold Max): 16685.1865 	 2.077s
CatBoost (3-fold Avg): 14736.1507
CatBoost (3-fold Max): 15103.5523 	 11.909s

Overall (Avg):         16347.916
Overall (Max):         17902.8628


# Principal Component Analysis

In [17]:
# Performs PCA on the whole dataframe
def pca_transform(X_train, X_valid, X_test, 
                  features = ["GarageArea","YearRemodAdd","TotalBsmtSF","GrLivArea"], 
                  n_components = 2):
    
    # Normalize based on training data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train[features])
    X_valid_scaled = scaler.transform(X_valid[features])
    X_test_scaled = scaler.transform(X_test[features])
    
    # Create principal components
    pca = PCA(n_components)
    X_pca = pca.fit_transform(X_scaled)
    X_valid_pca = pca.transform(X_valid_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    
    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names)
    X_valid_pca = pd.DataFrame(X_valid_pca, columns=component_names)
    X_test_pca = pd.DataFrame(X_test_pca, columns=component_names)
    
    return X_train.join(X_pca), X_valid.join(X_valid_pca), X_test.join(X_test_pca)

def test_pca_features():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX)
    
    transforms = [preprocessing, partial(pca_transform, n_components = 1)]
    print("\nPCA (1 components):\n")
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
   
    
    transforms = [preprocessing, partial(pca_transform, n_components = 2)]
    print("\nPCA (2 components):\n")
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
   
    
    transforms = [preprocessing, partial(pca_transform, n_components = 3)]
    print("\nPCA (3 components):\n")
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
   
    
test_pca_features()


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

PCA (1 components):

XGBoost  (3-fold Avg): 17325.0409
XGBoost  (3-fold Max): 18070.6054 	 1.905s
LightGBM (3-fold Avg): 16144.3925
LightGBM (3-fold Max): 16545.4439 	 1.858s
CatBoost (3-fold Avg): 14844.5589
CatBoost (3-fold Max): 15355.6194 	 10.502s

Overall (Avg):         16104.6641
Overall (Max):         17325.0409

PCA (2 components):

XGBoost  (3-fold Avg): 17173.8727
XGBoost  (3-fold Max): 18136.1141 	 1.94s
LightGBM (3-fold Avg): 16096.8338
LightGBM (3-fold Max): 16434.0747 	 1.826s
CatBoost (3-fold Avg): 14811.3066
CatBoost (3-fold Max): 15259.5257 	 11.896s

Overall (Avg):         16027.3377
Overall (Max):         17173.8727

PCA (3 components):

XGBoost  (3-fold Avg): 17227.3685
XGBoost  (3-fold Max): 18312.4517 	 1.939s
LightGBM (3-fold Avg): 16300.9474
LightGBM (3-fold Max): 16790.1549 	 1.861s
CatBoost (3-fold Avg): 14897.8518
CatBoost (3-fold Max): 15515.0018 	 11.524s

Overall (Avg):       

# Target Encoding

In [18]:
class CrossFoldEncoder:
    def __init__(self, encoder, **kwargs):
        self.encoder_ = encoder
        self.kwargs_ = kwargs  # keyword arguments for the encoder
        self.cv_ = KFold(n_splits=5)

    # Fit an encoder on one split and transform the feature on the
    # other. Iterating over the splits in all folds gives a complete
    # transformation. We also now have one trained encoder on each
    # fold.
    def fit_transform(self, X, y, cols):
        self.fitted_encoders_ = []
        self.cols_ = cols
        X_encoded = []
        for idx_encode, idx_train in self.cv_.split(X):
            fitted_encoder = self.encoder_(cols=cols, **self.kwargs_)
            fitted_encoder.fit(
                X.iloc[idx_encode, :], y.iloc[idx_encode],
            )
            X_encoded.append(fitted_encoder.transform(X.iloc[idx_train, :])[cols])
            self.fitted_encoders_.append(fitted_encoder)
        X_encoded = pd.concat(X_encoded)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

    # To transform the test data, average the encodings learned from
    # each fold.
    def transform(self, X):
        from functools import reduce

        X_encoded_list = []
        for fitted_encoder in self.fitted_encoders_:
            X_encoded = fitted_encoder.transform(X)
            X_encoded_list.append(X_encoded[self.cols_])
        X_encoded = reduce(
            lambda x, y: x.add(y, fill_value=0), X_encoded_list
        ) / len(X_encoded_list)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

In [19]:
def encode_neighborhood(X_train, X_valid, X_test, y_train):
    encoder = CrossFoldEncoder(MEstimateEncoder, m=1)
    X1_train = encoder.fit_transform(X_train, y_train, cols=["Neighborhood"])
    X1_valid = encoder.transform(X_valid)
    X1_test = encoder.transform(X_test)
        
    return X_train.join(X1_train), X_valid.join(X1_valid), X_test.join(X1_test)

def test_neighborhood_encoding():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX) 
    
    transforms = [preprocessing, encode_neighborhood]
    print("\nTarget Encoding ('Neighborhood'):\n")
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
   
    
test_neighborhood_encoding()


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

Target Encoding ('Neighborhood'):

XGBoost  (3-fold Avg): 17457.4707
XGBoost  (3-fold Max): 18292.309 	 2.043s
LightGBM (3-fold Avg): 16597.4962
LightGBM (3-fold Max): 17185.1397 	 1.921s
CatBoost (3-fold Avg): 14639.1022
CatBoost (3-fold Max): 14970.7286 	 10.842s

Overall (Avg):         16231.3564
Overall (Max):         17457.4707


In [20]:
def encode_subclass(X_train, X_valid, X_test, y_train):
    encoder = CrossFoldEncoder(MEstimateEncoder, m=1)
    X1_train = encoder.fit_transform(X_train, y_train, cols=["MSSubClass"])
    X1_valid = encoder.transform(X_valid)
    X1_test = encoder.transform(X_test)
        
    return X_train.join(X1_train), X_valid.join(X1_valid), X_test.join(X1_test)

def test_subclass_encoding():
    
    print("\nBaseline\n")
    print("Overall (Avg):".ljust(22), BASELINE_AVG)
    print("Overall (Max):".ljust(22), BASELINE_MAX)    
    
    transforms = [preprocessing, encode_subclass]
    print("\nTarget Encoding ('SubClass'):\n")
    xgb_avg, xgb_max = score_xgboost(transforms)
    lgbm_avg, lgbm_max = score_lightgbm(transforms)
    cat_avg, cat_max = score_catboost(transforms)
    print("\nOverall (Avg):".ljust(23), round(np.mean([xgb_avg, lgbm_avg, cat_avg]), 4))
    print("Overall (Max):".ljust(22),round(np.max([xgb_avg, lgbm_avg, cat_avg]), 4))
   
    
test_subclass_encoding()


Baseline

Overall (Avg):         16484.2446
Overall (Max):         17818.5785

Target Encoding ('SubClass'):

XGBoost  (3-fold Avg): 18093.2523
XGBoost  (3-fold Max): 18547.7723 	 2.261s
LightGBM (3-fold Avg): 16692.4652
LightGBM (3-fold Max): 17203.6731 	 1.963s
CatBoost (3-fold Avg): 15037.7819
CatBoost (3-fold Max): 15669.5137 	 10.8s

Overall (Avg):         16607.8331
Overall (Max):         18093.2523


# Test Strategies

We test combinations of all the promising strategies using optuna


In [21]:
def feature_search():
    def objective(trial):
        math = trial.suggest_categorical('math', [True, False])
        count = trial.suggest_categorical('count', [True, False])
        group = trial.suggest_categorical('group', [True, False])
        cluster_dist = trial.suggest_categorical('cluster_dist', [True, False])
        pca = trial.suggest_categorical('pca', [True, False])
        target = trial.suggest_categorical('target', [True, False])
        drop_last = trial.suggest_categorical('drop_last', [True, False])
        
        transforms = [preprocessing]
        
        if math:
            transforms.append(mathematical_transformations)
        if count:
            transforms.append(count_porch_types)
        if group:
            transforms.append(group_transformation)
        if cluster_dist:
            transforms.append(generate_cluster_distances)
        if pca:
            transforms.append(pca_transform)
        if target:
            transforms.append(encode_neighborhood)
        if drop_last:
            transforms.append(remove_uninformative)

        scores =  np.array([score_xgboost(transforms = transforms)[0],
                            score_lightgbm(transforms = transforms)[0],
                            score_catboost(transforms = transforms)[0],])
        time.sleep(1)
        return scores.mean()
    
    search_space = dict(math = [True, False],
                        count = [True, False],
                        group = [True, False],
                        cluster_dist = [True, False],
                        pca = [True, False],
                        target = [True, False],
                        drop_last = [True, False])
    optuna.logging.set_verbosity(optuna.logging.DEBUG)
    study = optuna.create_study(sampler = GridSampler(search_space), 
                                direction="minimize")
    study.optimize(objective, n_trials=2**len(search_space))
    return study

study = feature_search()

[I 2021-09-07 19:19:35,334] A new study created in memory with name: no-name-c83d4d39-a1c8-4d81-afef-4e76cd8b8895


XGBoost  (3-fold Avg): 17573.9972
XGBoost  (3-fold Max): 18235.9336 	 4.153s
LightGBM (3-fold Avg): 16089.2872
LightGBM (3-fold Max): 16498.7109 	 3.893s
CatBoost (3-fold Avg): 14904.1363
CatBoost (3-fold Max): 15453.2451 	 12.357s


[I 2021-09-07 19:19:56,750] Trial 0 finished with value: 16189.140233333334 and parameters: {'math': False, 'count': True, 'group': False, 'cluster_dist': False, 'pca': True, 'target': False, 'drop_last': True}. Best is trial 0 with value: 16189.140233333334.


XGBoost  (3-fold Avg): 17579.3272
XGBoost  (3-fold Max): 18261.5149 	 4.726s
LightGBM (3-fold Avg): 16160.661
LightGBM (3-fold Max): 16761.8491 	 4.415s
CatBoost (3-fold Avg): 14637.9043
CatBoost (3-fold Max): 15191.5539 	 14.861s


[I 2021-09-07 19:20:21,773] Trial 1 finished with value: 16125.964166666667 and parameters: {'math': True, 'count': True, 'group': False, 'cluster_dist': True, 'pca': True, 'target': False, 'drop_last': True}. Best is trial 1 with value: 16125.964166666667.


...

[I 2021-09-07 20:00:56,387] Trial 125 finished with value: 15951.2581 and parameters: {'math': False, 'count': True, 'group': False, 'cluster_dist': False, 'pca': True, 'target': False, 'drop_last': False}. Best is trial 72 with value: 15708.617566666668.


XGBoost  (3-fold Avg): 17037.394
XGBoost  (3-fold Max): 17500.7787 	 2.543s
LightGBM (3-fold Avg): 16584.762
LightGBM (3-fold Max): 17185.5498 	 2.298s
CatBoost (3-fold Avg): 14066.064
CatBoost (3-fold Max): 14719.2971 	 11.044s


[I 2021-09-07 20:01:13,291] Trial 126 finished with value: 15896.073333333334 and parameters: {'math': True, 'count': True, 'group': True, 'cluster_dist': True, 'pca': False, 'target': True, 'drop_last': False}. Best is trial 72 with value: 15708.617566666668.


XGBoost  (3-fold Avg): 16885.8223
XGBoost  (3-fold Max): 17680.4788 	 2.206s
LightGBM (3-fold Avg): 16455.192
LightGBM (3-fold Max): 16680.2114 	 2.065s
CatBoost (3-fold Avg): 14348.006
CatBoost (3-fold Max): 14823.4012 	 11.12s


[I 2021-09-07 20:01:29,706] Trial 127 finished with value: 15896.3401 and parameters: {'math': False, 'count': True, 'group': False, 'cluster_dist': False, 'pca': True, 'target': True, 'drop_last': False}. Best is trial 72 with value: 15708.617566666668.


In [22]:
print("Best Options:", study.best_params)

Best Options: {'math': True, 'count': True, 'group': False, 'cluster_dist': True, 'pca': True, 'target': True, 'drop_last': False}


In [24]:
plot_parallel_coordinate(study)